In [11]:
import pandas as pd
from tqdm.notebook import tqdm
file_path = 'Book1.xlsx'

all_sheets = pd.read_excel(file_path, sheet_name=None)

In [12]:
all_sheets

{'Sheet1':    متغیر  مستقل x متغیر وابسته y
 0           price            ytm
 1          981060       0.199879
 2          985500         0.5239
 3          964000       0.261768
 4         1000000       0.192395
 5          820000       0.273655
 6          917000       0.302638
 7          940000       0.259974
 8         1000000       0.187628
 9         1000000        0.18782
 10        1000000         0.1924
 11         939990       0.224509
 12         987000       0.210879
 13         677210       0.270623
 14        1000000       0.187602
 15         974900       0.242328
 16         837310       0.270919
 17         627870       0.269039
 18        1000000       0.191155
 19        1000000       0.227059
 20        1000000       0.250416
 21         950000       0.560114
 22         652840       0.270554
 23        1000000        0.25023
 24         944000       0.260754
 25         978800       0.242562
 26        1000000       0.187957
 27         980000       0.285704
 28 

In [13]:
import pandas as pd
from tqdm.notebook import tqdm

def read_all_sheets_from_excel(path: str) -> dict:
    all_sheets = pd.read_excel(path, sheet_name=None)
    dataframes = {}
    for sheet_name, df in tqdm(all_sheets.items()):
        # Strip whitespace from column names
        # df = df.rename(columns=lambda x: x.strip() if isinstance(x, str) else x)
        # # Strip whitespace from cells
        # df = df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)
        dataframes[sheet_name] = df
    return dataframes

# Load the data from the Excel file
file_path = 'Book1.xlsx'
dataframes = read_all_sheets_from_excel(file_path)

# Print each dataframe name and head
for sheet_name, df in dataframes.items():
    print('Dataframe from sheet:', sheet_name)
    print(df.head())

  0%|          | 0/2 [00:00<?, ?it/s]

Dataframe from sheet: Sheet1
  متغیر  مستقل x متغیر وابسته y
0          price            ytm
1            NaN            NaN
2            NaN            NaN
3            NaN            NaN
4            NaN            NaN
Dataframe from sheet: Sheet1 (2)
  متغیر مستقل دوم x2 متغیر  مستقل اول x1 متغیر وابسته y
0          inventory               price            ytm
1                NaN                 NaN            NaN
2                NaN                 NaN            NaN
3                NaN                 NaN            NaN
4                NaN                 NaN            NaN
